In [1]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig
import transformers
import torch
from dotenv import load_dotenv
import os
import openai
import banking  # noqa: E402
from private_prompting import Prompter
load_dotenv(".env")

open_ai_key = os.environ.get("openai-key")


Download data and initialize a DuckDB instance. 

In [2]:
_ = banking.BankingData("https://tinyurl.com/jb-bank", "bank")
_.extract_to_csv()

# Loading in SQL extension
%reload_ext sql
# Initiating a DuckDB database named 'bank.duck.db' to run our SQL queries on
%sql duckdb:///bank.duck.db

Create table.

In [3]:
%sql CREATE OR REPLACE TABLE bank AS FROM read_csv_auto('bank_cleaned.csv', header=True, sep=',')

Running query in 'duckdb:///bank.duck.db'

Count
4521


In [4]:
# Extract column names
columns = %sql PRAGMA table_info('bank');
column_names = [row[1] for row in columns]

Running query in 'duckdb:///bank.duck.db'

<h1 align='center'>Prompts & Agents</h1>

<h3 align='center'>How to incorporate prompting into your Python scripts and expand their functionality through agents</h3>

<h4 align='center'>Laura Funderburk</h4>

<h4 align='center'>PyData Vancouver</h4>

<h2 align='center'>About me</h2>

* Developer Advocate @ Ploomber (sharing knowledge about tools to improve the data science workflow)

* Previously a data scientist (for-profit, not-for-profit sector)

* Deeply curious about generative AI, Large Language Models, with a focus on engineering and automation

* I use LLMs, prompting and agents to automate work tasks

<h2 align='center'>Talk at a glance</h2>

<h3 align='center'>Part I: Prompting (40 minutes)</h3>


1. LLMs use cases and tasks
2. The Generative AI project lifecycle 
3. Choosing the right LLM architecture
4. Key elements of prompting & prompting techniques
5. Prompting private LLMs (OpenAI API): `ChatCompletion`
6. Prompting open source LLMs through HuggingFace

<h2 align='center'>Talk at a glance</h2>

<h3 align='center'>Part II: Agents and open source frameworks (20 minutes)</h3>

1. What are agents
2. Introduction to Haystack
3. Introduction to LangChain
4. Techniques to combine prompting and agents for deployment of applications
5. Pros and cons of each

<h1 align='center'>Part I: Prompting</h1>


<h2 align='center'>LLMs use cases and tasks</h2>


* Text summarization

* Conversation 

* Translation

* Text generation

* Text, token and sentiment classification

* Table Q&A and Q&A from unstructured data

* Sentence similarity

* Masking

<h2 align='center'>LLMs use cases and tasks</h2>

<h3 align='center'>Your goal is to understand the business case you are solving - then select the appropriate methods to solve it</h3>

$\Rightarrow$ Who will benefit from your product?

$\Rightarrow$ What are business constraints (time, data, resources)?

$\Rightarrow$ What is the end result?

$\Rightarrow$ How will it be served?



<h2 align='center'>The generative AI project lifecycle</h2>
<h3 align='center'>Focus of this talk</h3>


<p></p>
<center>
  <img src="diagrams/genai_project_lifecycle_focus.jpg" width="1200px"/>

</center>

Source: Coursera, Generative AI with LLMs

<h2 align='center'>Choosing the right LLM (architecture)</h2>

<p></p>
<center>
  <img src="diagrams/opt.jpeg" width="200px"/>

</center>


<p></p>

| Tranformer type | Architecture|Model-like | Focus | Example| 
|-|-|-|-|-|
| Auto-regressive | Decoder-only |GPT-like | Generative tasks | Chat bot | 
| Auto-encoding | Encoder-only |BERT-like | Understanding of the input | Question-answering|
| Sequence-to-Sequence |Encoder-decoder |BART/T5-like | Generative tasks that require an input | Language translation|


<p></p>

“Attention is all you need” (Vaswani, Ashish & Shazeer, Noam & Parmar, Niki & Uszkoreit, Jakob & Jones, Llion & Gomez, Aidan & Kaiser, Lukasz & Polosukhin, Illia. (2017))

[https://github.com/christianversloot/machine-learning-articles](https://github.com/christianversloot/machine-learning-articles/blob/main/differences-between-autoregressive-autoencoding-and-sequence-to-sequence-models-in-machine-learning.md)

<h2 align='center'>Do I need to train a new model to solve my problem?</h2>

**No. Training an LLM is costly (GPU usage, time, compute, data). This is why sharing LLMs and their fine-tuned components has become highly popular.**


<p></p>
<center>
  <img src="diagrams/hftasks.png" width="1200px"/>

</center>

You can start with prompting a LLM, then fine-tuning* or using retrieval augmentation if you aren't getting the results you want. You'll need to curate a dataset for this.

*(instruction-tuning or, PEFT + LoRA for example)

Source: https://huggingface.co/tasks

<h1 align='center'>Prompting</h1>


<h2 align='center'>Key elements of prompting</h2>

<h3 align='center'>Basic</h3>

<p></p>
<center>
  <img src="diagrams/tokens.png" width="1200px"/>

</center>

* An LLM to interact with
* Temperature
* Max tokens
* A natural language request



<h2 align='center'>Key elements of prompting</h2>


<h3 align='center'>Advanced</h3>

* Data (text files, web files)
* A database storage system (vector DB, SQL, PostgreSQL, etc)
* User interfaces


<h2 align='center'>Prompting techniques</h2>

* Zero-shot inference

* One-shot inference

* Few-shot inference

* Chain of thought 

* Roles (OpenAI API)

<h2 align='center'>Prompting private LLMs (OpenAI API)</h2>


We're going to focus on the `ChatCompletion` end point. 

Key elements: OpenAI API Key, model (GPT4, GPT 3.5 Turbo, Text-Davinci), temperature, prompt, max tokens



<p></p>
<center>
  <img src="diagrams/chat-completion.png" width="800px"/>

</center>





<h3 align='center'>Prompting techniques: Zero-shot inference</h3>

**Formula: instruction, no examples.**

Suppose we want to translate a natural language question to SQL.

```python
prompt = f"Answer the question {natural_question} \
           for table {db_name} \
           with schema {schema}"
```

<h3 align='center'>Prompting techniques: One-shot inference</h3>

**Formula: instruction, one example.**


```python
prompt = f"Answer the question {natural_question} \
           for table {db_name} \
           with schema {schema}\
               Question: How many records are there?\
               Answer: SELECT COUNT(*) FROM bank"

```

<h3 align='center'>Prompting techniques: Few-shot inference</h3>

**Formula: instruction, more than one example.**

```python
prompt= f"Answer the question {natural_question} \
           for table {db_name} \
           with schema {schema}\
            Question: How many records are there?\
            Answer: SELECT COUNT(*) FROM bank\
            Question: Find all employees that are unemployed\
            Answer: SELECT * FROM bank WHERE job = 'unemployed'"
```

<h3 align='center'>Roles in prompting the ChatCompletion endpoint (OpenAI API only)</h3>

The 'role' can take one of three values: `system`, `user` or the `assistant`

The `content` contains the text of the message from the role. 

`system` role: You can use a system level instruction to guide your model's behavior throughout the conversation. 

    You are a helpful ... with knowledge about ...

`user` role: What are typical requests that someone in that role would receive?

    Answer the question ... 

`assistant` role: This role  represents the language model, such as ChatGPT, which generates responses based on the provided user messages.


<h2 align='center'>Business problem: translate natural language questions into SQL</h2>

We can solve this problem with prompting and the `ChatCompletion` endpoint on the OpenAI API.

**Approach: build a Prompter class and add each prompting technique as a method, then evaluate results**

<h3 align='center'>Approach: initialize a Prompter class</h3>

```python
import openai

class Prompter:
    def __init__(self, api_key, gpt_model, temperature=0.2):
        if not api_key:
            raise Exception("Please provide the OpenAI API key")

        self.api_key  = api_key
        self.gpt_model = gpt_model
        self.temperature = temperature
    
    
```
    

<h3 align='center'>Approach: add a chat completion method to call a GPT-like model (OpenAI API)</h3>




<p></p>
<center>
  <img src="diagrams/init-chatcompletion.png" width="1200px"/>

</center>

<h3 align='center'>Approach: add a method with a single-shot prompt and the assistant role</h3>


<p></p>
<center>
  <img src="diagrams/prompt-roles-assistant.png" width="1200px"/>

</center>


<h3 align='center'>Approach: add a method with a system and user roles</h3>

<p></p>
<center>
  <img src="diagrams/prompt-roles-sql.png" width="1200px"/>

</center>


<h2 align='center'>Evaluate results</h2>


Let's suppose I have a DuckDB in-memory instance with a table called `bank` that looks as follows.

In [5]:
%sqlcmd explore --table bank

<h2 align='center'>Evaluate results</h2>

Let's take the different prompting techniques for a ride.

We will ask the OpenAI API GPT-3.5-turbo model to translate a natural language question into SQL. 

In [6]:
pm  = Prompter(open_ai_key, "gpt-3.5-turbo")

Zero-shot results.

In [7]:
pm.natural_language_zero_shot("bank", 
                              column_names, 
                              "How many unique jobs are there?")

"To determine the number of unique jobs in the table, we need to look at the 'job' column. We can use the DISTINCT keyword in SQL to get the unique values in that column. Here's an example query:\n\nSELECT COUNT(DISTINCT job) AS unique_jobs\nFROM bank;\n\nThis query will return the number of unique jobs in the 'job' column of the 'bank' table."

In [8]:
pm.natural_language_zero_shot("bank", 
                              column_names, 
                              "What is the total balance for \
                               employees by education?")

'To find the total balance for employees by education, we need to group the data by education and sum the balance column. Here is the SQL query to achieve this:\n\nSELECT education, SUM(balance) AS total_balance\nFROM bank\nGROUP BY education;'

Single-shot results.

In [9]:
pm.natural_language_few_shot("bank", 
                                column_names, 
                                "How many unique jobs are there?")

'Question: How many unique jobs are there?\nAnswer: SELECT COUNT(DISTINCT job) FROM bank'

In [10]:
pm.natural_language_few_shot("bank", 
                                column_names, 
                                "What is the total balance for \
                                employees by education?")

'Answer: The total balance for employees by education cannot be determined without further information.'

Roles-based results.

In [11]:
pm.natural_language_with_roles("bank", 
                               column_names, 
                               "How many unique jobs are there?")

'SELECT COUNT(DISTINCT job) FROM bank'

In [12]:
pm.natural_language_with_roles("bank", 
                               column_names, 
                               "What is the total balance for\
                               employees by education?")

'SELECT education, SUM(balance) AS total_balance\nFROM bank\nGROUP BY education'

In [13]:
%%sql
SELECT education, SUM(balance) AS total_balance 
FROM bank GROUP BY education;

Running query in 'duckdb:///bank.duck.db'

education,total_balance
primary,957027
secondary,2759854
tertiary,2396822
unknown,318133


<h3 align='center'>Prompting open source LLMs through HuggingFace</h3>


You need to ensure you install the right modules via `pip` along with any modules specified in the model card of the LLM.

<p></p>
<center>
  <img src="diagrams/hftasks.png" width="800px"/>

</center>


Example: https://huggingface.co/microsoft/tapex-base

<h3 align='center'>The reality of prompting open source models</h3>

<p></p>
<center>
  <img src="diagrams/this-is-fine.jpeg" width="600px"/>

</center>


<h3 align='center'>The reality of prompting open source models</h3>

* HuggingFace hosted models resemble GitHub repos (but not in a good way).
* You will need to feel comfortable using the `transformers`, `PyTorch` and `TensorFlow` libraries.
* You will need a bit more than just comfort with the transformer architectures.
* Dependency hell.
* Prompting results vary across different models.
* Model documentation ranges from non-existent to highly technical (research papers).
* Higher likelihood that you'll need to find the base model and fine-tune with your data for better results.


<h3 align='center'>Prompting a T5-like model to translate NL to SQL</h3>

We will explore the functionality of fine-tuned T5 model via the id 

`mrm8488/t5-base-finetuned-wikiSQL`

Remember that T5-like models are of type encoder-decoder and good at translating between languages.

This model was fine-tuned on the `wiki-SQL` dataset.

**Note that this model was not instruction-tuned**

<p></p>
<center>
  <img src="diagrams/t5-tuned.png" width="1000px"/>

</center>

In [14]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
/Users/macpro/anaconda3/envs/llm-env/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:1362: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(
The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


In [15]:
def get_sql(query):
    input_text = "translate English to SQL: %s </s>" % query
    features = tokenizer([input_text], 
                         return_tensors='pt')

    output = model.generate(input_ids=features['input_ids'], 
                           attention_mask=features['attention_mask'],
                            max_new_tokens=200)

    return tokenizer.decode(output[0])

# Translate
natural_question = "How many entries are there?" 
db_name = "banks"
schema = column_names

prompt = f"{natural_question} \
           for table {db_name} \
           with schema {schema}"

get_sql(prompt)

"<pad> SELECT COUNT Table FROM table WHERE Schema = ['age', 'job','marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day','month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y</s>"

<h3 align='center'>How to guide your choices</h3>

1. Remember your use case, the business constraints and who will use your application
2. Remember the three base models and their keywords
3. Choose **instruction-tuned foundation models** as a starting point for fine-tuning or retrieval augmentation.

<p> </p>

| Tranformer type | Architecture|Model-like | Focus | Example| 
|-|-|-|-|-|
| Auto-regressive | Decoder-only |GPT-like | Generative tasks | Chat bot | 
| Auto-encoding | Encoder-only |BERT-like | Understanding of the input | Question-answering|
| Sequence-to-Sequence |Encoder-decoder |BART/T5-like | Generative tasks that require an input | Language translation|

<h1 align='center'>Part II: Agents and open source frameworks</h1>

We will now turn our attention to two open-source frameworks you can use to augment the functionality of prompting through agents: LangChain and Haystack. 

The frameworks introduced here can both be installed via `pip` and imported as modules into your Python script. 



<h2 align='center'>What are agents</h2>

The role of an agent is to empower LLMs to decide which actions to take, thereby granting them a certain degree of autonomy. In simple terms, agents are a fusion of LLM chains (which are sequences of LLMs) and tools.

<h2 align='center'>Introducing LangChain</h2>
<center>
  <img src="diagrams/langchain.png" width="300px"/>

</center>

LangChain is a framework for developing applications powered by language models. It enables applications that are:

* Data-aware: connect a language model to other sources of data

* Agentic: allow a language model to interact with its environment


<h2 align='center'>How does LangChain approach Agents?</h2>

<p> </p>

<center>
  <img src="diagrams/langchain.jpg" width="1200px"/>

</center>

With `LangChain` we think in terms of **components** and **off-the-shelf chains**.

<h2 align='center'>How to incorporate it into your scripts</h2>

You can build your custom functions in Python and use their `@tool` decorator.


| | |
|-|-|
|![](diagrams/scraping-tool.png)|![](diagrams/summarize-tool.png)|

<h2 align='center'>How to incorporate it into your scripts</h2>

Then after initializing LangChain along with the GPT model you want, you can then ask it to perform tasks with natural language commands. 

<p> </p>

<center>
  <img src="diagrams/LangChain-use.png" width="1000px"/>

</center>


<h2 align='center'>Introducing Haystack</h2>
<center>
  <img src="diagrams/haystack-ogimage.png" width="500px"/>

</center>

Haystack is an open-source framework for building search systems that work intelligently over large document collections.



Functionality:

- Call open source models, hosted models (Azure, AWS) as well as private ones (OpenAI API)
- Build production-ready NLP pipelines with their custom-built tools
- Machinery for unstructured data processing (text)
- Leverage their prompt templates ([prompt-hub](https://prompthub.deepset.ai/))
- Incorporate Agents
- Compatibility with Vector and classic DB.
- Deploy via REST API

<h2 align='center'>How does Haystack approach Agents?</h2>



**Nodes**: each Node achieves one thing 

**Pipelines**: this is the standard Haystack structure that can connect to your data and perform on it NLP tasks that you define. 

**Document stores**: designed to make document retrieval and processing easier, can be connected to nodes via pipelines.

**Tools**: you can think of a Tool as an expert, that is able to do something really well.

**Agent**: a component that is powered by an LLM, such as GPT-3. It can use tools and decide on the next best course of action so as to get to the result of a query. 



<h2 align='center'>Creating a custom node to perform SQL queries in Jupyter</h2>

We are going to use `JupySQL` to perform the quries. 

`JupySQL` was developed on top of iPython-SQL and its purpose is to connect to DBs of various flavours and execute queries. 

This will be our **tool** (`JupySQLQuery`) and we will define it as a subclass of the `BaseComponent` class in Haystack.

We will also create a prompt with detailed instructions for how the agent should respond to different situations. 


Initialize JupySQL - this is a tool that can execute queries from Jupyter via the `%sql` and `%%sql` magics.

```python
from haystack.nodes.base import BaseComponent

class JupySQLQuery(BaseComponent):
    outgoing_edges = 1
    
    def __init__(self):
        %reload_ext sql
        %sql duckdb:///bank.duck.db
```

Add a method to execute one query via the `%sql` magic.

```python
from haystack.nodes.base import BaseComponent

class JupySQLQuery(BaseComponent):
    ...
    def run(self, query: str):
        result = %sql {{query}}
        output = {
            "results":  f"{result}",
            "query": query,
            
        }
        return output

```

Add a method to execute a list of queries via the `%sql` magic.


```python
from haystack.nodes.base import BaseComponent

class JupySQLQuery(BaseComponent):
    ...
    def run_batch(self, queries: list):
        results = []
        for query in queries:
            result = %sql {query}
            output = {
                "results":  f"{result}",
                "query": query,
            }
            results.append(output)
        return results

```

In [16]:
from haystack.nodes.base import BaseComponent

class JupySQLQuery(BaseComponent):
    outgoing_edges = 1
    
    def __init__(self):
        %reload_ext sql
        %sql duckdb:///bank.duck.db

    def run(self, query: str):
        result = %sql {{query}}
        output = {
            "results":  f"{result}",
            "query": query,
            
        }
        return output

    def run_batch(self, queries: list):
        results = []
        for query in queries:
            result = %sql {query}
            output = {
                "results":  f"{result}",
                "query": query,
            }
            results.append(output)
        return results

    
jupy_sql_query = JupySQLQuery()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
from haystack.agents import Tool
from haystack.nodes import PromptNode
from jupysqlagent import sql_agent_prompt
from haystack.agents import Agent, Tool


jupy_sql_query = JupySQLQuery()
jupy_sql_query_tool = Tool(name="JupySQL_Query", 
                           pipeline_or_node=jupy_sql_query, 
                           description="""This tool is useful for consuming SQL queries \
                                        and responds with the result""")


In [18]:
# Get the API key
openai_api_key = os.environ.get("openai-key")
chosen_model = "gpt-4"


# Define a prompt node that uses the GPT-4 model
prompt_node = PromptNode(model_name_or_path=chosen_model, 
                         api_key=openai_api_key, 
                         stop_words=["Observation:"], 
                         max_length=1000)

# Define the agent
agent = Agent(prompt_node=prompt_node, 
              prompt_template=sql_agent_prompt)

agent.add_tool(jupy_sql_query_tool)

In [ ]:
result = agent.run("How many records are there")

In [ ]:
result = agent.run("How many unique levels of education are there")

<h3 align='center'>LangChain Pros & Cons</h3>

**Pros**
1. Easy to get started with
2. Maps easily to OpenAI API chat completion end point
3. Can easily connect to a variety of applications based on your function definition

**Cons**
1. Security concerns
2. Evaluation of results 
3. Deployment 
4. Integration to open LLMs and hosted LLMs seems to be in early stages

<h3 align='center'>Haystack Pros & Cons</h3>

**Pros**
1. Established framework with a focus on production-ready NLP applications
2. Constantly adapting to new changes and building on top of their framework
3. Deployment-friendly
4. Offers solutions for your custom documents and access to a variety of database flavours
5. Offers prompt templates

**Cons**
1. Steeper learning curve
2. Current deployment option is REST API, but other options currently not available
3. Limitations on the types of files it can handle (PDF and markdown currently not supported) 
4. Narrower focus when it comes to the types of agents it supports (although you can create custom agents, through custom nodes)

<h1 align='center'>Final thoughts</h1>

* Prompt-engineering starts with a well defined project and a clear choice of transformer architecture
* Prompting is usually the first step when using an LLM
* Prompting via the OpenAI API provides a quick solution to prototype, but has limitations when it comes to private data/documents
* Prompting open source LLMs requires understanding of transformer architecture and openness to fine-tune 
* We explored two open source frameworks that allow you to augment the funcionality of LLMs via agents
* LangChain approaches agents through **components** and **chains**
* Haystack approaches agents in terms of expanding the funcionality of **prompt nodes**, **pipelines** and a **document store**.